In [126]:
import json, urllib, csv, locale, collections
## import glob, os
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time, random, requests

In [128]:
drinkByName = dict()
"""
with open('../app/static/drinks.json', 'r') as infile:  
    jsonData = json.load(infile)
    drinklst = jsonData['drinks']

for drink in drinklst:
    drinkByName[drink['name']] = drink
"""

dict_keys(['A to Z Kamikaze', 'Apple Cider Float', 'Cran-Kamikaze', 'Amaretto Pina Colada', "Maker's Mark® Manhattan", 'Appletini', 'Naughty List Hot Chocolate', 'Classic Margarita', 'Sunset Daiquiri', 'Naked Cosmo'])


"\nwith open('../app/static/drinks.json', 'r') as infile:  \n    jsonData = json.load(infile)\n    drinklst = jsonData['drinks']\n\nfor drink in drinklst:\n    drinkByName[drink['name']] = drink\n"

In [5]:
print(len(drinkByName))

0


In [132]:
def scrape_fmt_1(soup):
    # "https://www.thecocktailproject.com/drink-recipes/peach-sangria"
    
    drink = dict()
    
    name = soup.find('section', attrs={'class':'recipe-info'})
    name = name.find('h1', attrs={'itemprop':'name'})
    drink['name'] = name.text.strip()
    
    #not used
    #desc = title.find('div', attrs={'class':'recipe-description hidden-xs'})
    #drink['description'] = desc.text
    
    flavor = soup.find_all('div', attrs={'class':'drink-properties-incredible'})[1]
    flavor = flavor.find('p')
    drink['flavors'] = [a.strip() for a in flavor.text.split(',')]
    
    ing = soup.find_all('li', attrs={'itemprop':'ingredients'})
    drink['ingredients'] = [i.text.strip() for i in ing]
    
    return drink

In [143]:
def scrape_fmt_2(soup):
    drink = dict()
    
    name = soup.find('h1', attrs={'class':'text-uppercase','itemprop':'name'})
    drink['name'] = name.text.strip()
    
    #not used
    #desc = title.find('div', attrs={'class':'recipe-description hidden-xs'})
    #drink['description'] = desc.text
    
    flavor = soup.find('div', attrs={'class':'drink-properties row'})
    flavor = flavor.find_all('p')[1]
    drink['flavors'] = [a.strip() for a in flavor.text.split(',')]
    
    ing = soup.find_all('li', attrs={'itemprop':'ingredients'})
    drink['ingredients'] = [i.text.strip() for i in ing]
    
    return drink

session = requests.Session()
r = session.get("https://www.thecocktailproject.com/drink-recipes/makers-mark-eggnog", cookies=cookies)
soup = BeautifulSoup(r.text, 'html.parser')
print(scrape_fmt_2(soup))

{'ingredients': ["1 liter Maker's Mark® Bourbon", '1 quart Milk', '1 quart Heavy Cream', '2 dozen Eggs', '1 1/2 cups Sugar', 'Garnish Nutmeg'], 'flavors': ['Sweet'], 'name': "Maker's Mark® Eggnog"}


In [147]:
cookies = dict(cookie="ARRAffinity=dcad1d885cffc135d97656b8bc685a8639350bc655376093305c49265b0ed09a; has_js=1; _gcl_au=1.1.1422552885.1556300604; _fbp=fb.1.1556300603544.757912343; _ga=GA1.2.441517185.1556300604; _gid=GA1.2.773503897.1556300604; _gat_UA-106357112-8=1; _gat_UA-106357112-12=1; age-gate=true; age-gate-year=1997; age-gate-age=21")

def drink_scraper(url):
    drink = dict()
    
    session = requests.Session()
    
    r = session.get(url, cookies=cookies)
    
    soup = BeautifulSoup(r.text, 'html.parser')
    
    try:
        drink = scrape_fmt_1(soup)
    except AttributeError:
        drink = scrape_fmt_2(soup)
    
    return drink
    
    
print(drink_scraper("https://www.thecocktailproject.com/drink-recipes/peach-sangria"))
print(drink_scraper("https://www.thecocktailproject.com/drink-recipes/makers-mark-eggnog"))

{'ingredients': ['1 part DeKuyper® Peachtree® Schnapps Liqueur', '3 parts White Wine', 'Fruit, Chopped', 'Splash of Soda Water'], 'flavors': ['Fruity', 'Sweet'], 'name': 'Peach Sangria'}
{'ingredients': ["1 liter Maker's Mark® Bourbon", '1 quart Milk', '1 quart Heavy Cream', '2 dozen Eggs', '1 1/2 cups Sugar', 'Garnish Nutmeg'], 'flavors': ['Sweet'], 'name': "Maker's Mark® Eggnog"}


In [ ]:
urls = dict()

In [176]:
def scrape_urls():
    url = "https://www.thecocktailproject.com/drink-recipes"
    
    i = 0
    while True:
        print(i)
        session = requests.Session()
        r = session.post(url, cookies=cookies, params=dict(page=i))

        soup = BeautifulSoup(r.text, 'html.parser')
        soup = soup.find('div', attrs={'class':'main-content-wrapper'})

        drinkLinks = soup.find_all('div', attrs={'class':'grid-main-container-col col-md-4 col-sm-6 col-xs-12 animation fadeInUp animation-visible'})

        for d in drinkLinks:
            u = d.find('a', href=True)
            u = "https://www.thecocktailproject.com"+u['href']
            
            urls[u] = True
            
        print(len(urls))
        i += 1
        r.connection.close()

def scrape_pages():
    for url in urls.keys():
        if urls[url] != True:
            continue
            
        time.sleep(random.randint(2,5))
            
        try:
            d = drink_scraper(url)
            drinkByName[d['name']] = d
            print(d['name'])
            urls[url] = False
            
        except AttributeError:
            print("FAILED ON "+url)
    
scrape_pages()

Jim Beam Black® Manhattan
Red Cherry Bonbon Shot
Picnic Punch
Bare Naked Martini
Brass Monkey
Cognac Summit
Bloody Mark
Fuzzy Beam
Cruzan® Banana Split
Sherbet Sunrise
Cruzan® Single Barrel on the Rocks
Rouge Margarita
DeKuyper® Banana Split
Watermelon Rush
EFFEN® Blood Orange Mimosa
Red, White, and Blue Shot
The Raspberry Julep
Sparkling Cherry Sangria
Buttery Nipple
Cruzan® 9 and Ginger Beer
Key Lime Pie
Strawberry Tease Martini
Holly Jolly Punch
Bloody Sunrise
Cocoa Candy Cane
Blue Hawaii
Mango Cran-Tini
Black & Gold
The Chee Chee
Kiwi Zest Martini
The Money Maker
Midori® Orange and Sparkling
Chocolate Martini
Bloody Maria
Cucumber Cooler
Handsome Ginger
Classic Old Fashioned
Gimlet
Sauza® Strawberry Sparkler
Midori® Splice
Chardonnay Spritzer
Cruzan® Daiquiri
Big Ginger®
Cruzan® Summer Breeze
Classic Lemon Drop
Peachy Keen
Ginger Cider
Peanut Butter Cup Martini
Black Beauty
Bellini Manhattan
Sparkling Cognac
Sour Apple Popsicle Martini
Smore-Tini
Apple Daiquiri
Weekenders' Punch
Th

Devil in the Morning
Cruzan® Strawberry & Ginger Ale
Passionway Daiquiri
Marshmallow Float
French 75
Gin & Tonic
Zombie
Limonlicious
Melon Ball
Red Sangria Pitcher
Cruzan® Confusion
Citrus Fizz
Cookies and Cream
Watermelon Mai Tai
Minted Joe
Pinta Colada
Deluxe Mojito
The Brown Derby
Beam® Ade
Cookie-Tini
Amaretto Coffee
Citrus Basil Manhattan
Jim Beam® Lemonade
Pinnacle® Cherry Cola
Hurricane
Sauza® Strawberry Margarita
Bloody Mary
Shark's Tooth
Carpe Diem
Elderflower Margarita
Endless Summer Punch
Maker's Mark® Manhattan
Jim Beam® Vanilla & Cola
Snickerdoodle Cookie Martini
Sideline Sipper
Dragons Toe
Melon Berry Mist
Peppermint Hot Cocoa
Vodka Collins
Peach Sangria
Courvoisier® Old Fashioned
New England Apple Pie
Countdown Punch
Red Greyhound Pinn-Tini
Strawberry Julep
Rum Breeze
Black Lemon Razzmatazz®
Fresh Midori® Sour
Cruzan® Sidecar
Glasgow Seelbach
Sloe Gin Fizz
Honey King
Pumpkin Spice White Russian
Raspberry Whiskey Sour
Game Day Peach Sour
French Martini
CC® and Ginger
Pool

In [175]:
print(drinkByName['Paris Escape'])

{'ingredients': ['1 part Pinnacle® Original Vodka', '1 part Pineapple Juice', 'Raspberries, Muddled', 'Lemon', 'Raspberries'], 'flavors': ['Fruity'], 'name': 'Paris Escape'}


In [157]:
jsondata = {}
jsondata['drinks'] = []
for drink in drinkByName.values():
    
    jsondata['drinks'].append({
        'name' : drink['name'],
        'ingredients' : drink['ingredients'],
        'flavors' : drink['flavors']
    })

In [158]:
with open('./flavor-data.json', 'w') as outfile:  
    json.dump(jsondata, outfile)